Performance Marketing Attribution with Conversion Lag

This notebook models delayed customer conversions in performance marketing using lagged spend variables and rolling ROAS. The goal is to improve budget allocation decisions by capturing the true impact of upper-funnel and influencer campaigns.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)


In [ ]:
dates = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")


In [ ]:
dates = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")

channels = [
    "Google Search",
    "Google Display",
    "Meta",
    "Influencer",
    "Affiliate",
    "Email"
]


In [ ]:
channel_params = {
    "Google Search": {"base_spend": 740000, "ctr": 0.045, "cvr": 0.06, "aov": 1200},
    "Google Display": {"base_spend": 410000, "ctr": 0.015, "cvr": 0.02, "aov": 1000},
    "Meta": {"base_spend": 980000, "ctr": 0.025, "cvr": 0.035, "aov": 1100},
    "Influencer": {"base_spend": 490000, "ctr": 0.02, "cvr": 0.03, "aov": 1300},
    "Affiliate": {"base_spend": 330000, "ctr": 0.03, "cvr": 0.04, "aov": 1150},
    "Email": {"base_spend": 250000, "ctr": 0.06, "cvr": 0.08, "aov": 1400}
}


In [ ]:
missing_channels = set(channels) - set(channel_params.keys())
if missing_channels:
    raise ValueError(f"Missing parameters for channels: {missing_channels}")


In [ ]:
records = []

for date in dates:
    for channel in channels:
        params = channel_params[channel]

        spend = max(np.random.normal(
            params["base_spend"], params["base_spend"] * 0.1
        ), 0)

        impressions = spend * np.random.uniform(8, 12)
        clicks = impressions * params["ctr"] * np.random.uniform(0.9, 1.1)
        conversions = clicks * params["cvr"] * np.random.uniform(0.9, 1.1)
        revenue = conversions * params["aov"] * np.random.uniform(0.95, 1.05)

        records.append([
            date,
            channel,
            round(spend, 2),
            int(impressions),
            int(clicks),
            int(conversions),
            round(revenue, 2)
        ])

marketing_performance = pd.DataFrame(
    records,
    columns=[
        "date", "channel", "spend",
        "impressions", "clicks",
        "conversions", "revenue"
    ]
)


In [ ]:
customer_records = []

for _, row in marketing_performance.iterrows():
    new_ratio = np.random.uniform(0.55, 0.75)

    new_customers = int(row["conversions"] * new_ratio)
    returning_customers = row["conversions"] - new_customers

    customer_records.append([
        row["date"],
        row["channel"],
        new_customers,
        returning_customers
    ])

customer_type = pd.DataFrame(
    customer_records,
    columns=["date", "channel", "new_customers", "returning_customers"]
)


In [ ]:
product_records = []

for _, row in marketing_performance.iterrows():
    acne_ratio = np.random.uniform(0.35, 0.45)
    pigmentation_ratio = np.random.uniform(0.25, 0.35)
    hydration_ratio = 1 - (acne_ratio + pigmentation_ratio)

    product_records.append([
        row["date"],
        row["channel"],
        int(row["conversions"] * acne_ratio),
        int(row["conversions"] * pigmentation_ratio),
        int(row["conversions"] * hydration_ratio)
    ])

product_category = pd.DataFrame(
    product_records,
    columns=["date", "channel", "acne", "pigmentation", "hydration"]
)


In [ ]:
print("Rows:", marketing_performance.shape)
print("Total Spend (₹ Cr):", round(marketing_performance["spend"].sum() / 1e7, 2))
print("Overall ROAS:", round(
    marketing_performance["revenue"].sum() /
    marketing_performance["spend"].sum(), 2
))


Rows: (2196, 7)
Total Spend (₹ Cr): 116.8
Overall ROAS: 18.68


In [ ]:
marketing_performance.to_csv("marketing_performance.csv", index=False)
customer_type.to_csv("customer_type.csv", index=False)
product_category.to_csv("product_category.csv", index=False)


In [ ]:
import pandas as pd

marketing = pd.read_csv("marketing_performance.csv", parse_dates=["date"])


In [ ]:
marketing = marketing.sort_values(by=["channel", "date"])


In [ ]:
marketing["ctr"] = marketing["clicks"] / marketing["impressions"]
marketing["cvr"] = marketing["conversions"] / marketing["clicks"]
marketing["cpc"] = marketing["spend"] / marketing["clicks"]
marketing["cac"] = marketing["spend"] / marketing["conversions"]
marketing["roas"] = marketing["revenue"] / marketing["spend"]


In [ ]:
marketing.replace([float("inf"), -float("inf")], pd.NA, inplace=True)


In [ ]:
for lag in [1, 3, 7]:
    marketing[f"spend_lag_{lag}d"] = (
        marketing
        .groupby("channel")["spend"]
        .shift(lag)
    )


In [ ]:
marketing["spend_7d_avg"] = (
    marketing
    .groupby("channel")["spend"]
    .rolling(7)
    .mean()
    .reset_index(level=0, drop=True)
)

marketing["conversions_7d_avg"] = (
    marketing
    .groupby("channel")["conversions"]
    .rolling(7)
    .mean()
    .reset_index(level=0, drop=True)
)

marketing["roas_7d_avg"] = (
    marketing
    .groupby("channel")["roas"]
    .rolling(7)
    .mean()
    .reset_index(level=0, drop=True)
)


In [ ]:
marketing[[
    "date", "channel", "spend", "roas",
    "spend_lag_1d", "spend_lag_7d", "roas_7d_avg"
]].head(10)


,date,channel,spend,roas,spend_lag_1d,spend_lag_7d,roas_7d_avg
4,2024-01-01,Affiliate,300035.21,13.489154,NaN,NaN,NaN
10,2024-01-02,Affiliate,326183.61,13.217702,300035.21,NaN,NaN
16,2024-01-03,Affiliate,341933.99,16.252215,326183.61,NaN,NaN
22,2024-01-04,Affiliate,306832.25,13.354683,341933.99,NaN,NaN
28,2024-01-05,Affiliate,328854.51,16.591683,306832.25,NaN,NaN
34,2024-01-06,Affiliate,356846.07,15.407704,328854.51,NaN,NaN
40,2024-01-07,Affiliate,368233.65,16.251026,356846.07,NaN,14.937738
46,2024-01-08,Affiliate,302055.81,14.240207,368233.65,300035.21,15.045031
52,2024-01-09,Affiliate,313983.00,13.259543,302055.81,326183.61,15.051009
58,2024-01-10,Affiliate,290390.22,11.665349,313983.00,341933.99,14.395742


In [ ]:
baseline = (
    marketing
    .groupby("channel")
    .agg(
        total_spend=("spend", "sum"),
        total_revenue=("revenue", "sum"),
        total_conversions=("conversions", "sum"),
        avg_roas=("roas", "mean"),
        avg_cac=("cac", "mean")
    )
    .reset_index()
)


In [ ]:
baseline["spend_share_pct"] = (
    baseline["total_spend"] / baseline["total_spend"].sum()
) * 100


In [ ]:
baseline = baseline.sort_values(by="total_spend", ascending=False)
baseline


,channel,total_spend,total_revenue,total_conversions,avg_roas,avg_cac,spend_share_pct
5,Meta,3.574055e+08,3.454887e+09,3142973,9.670158,115.961618,30.599846
3,Google Search,2.697092e+08,8.739446e+09,7299653,32.411062,37.702663,23.091587
4,Influencer,1.797609e+08,1.393696e+09,1071356,7.771950,170.717573,15.390517
2,Google Display,1.492427e+08,4.480070e+08,448027,3.000629,340.083444,12.777652
0,Affiliate,1.200911e+08,1.644902e+09,1434453,13.695398,85.408097,10.281789
1,Email,9.178837e+07,6.134235e+09,4372993,66.795442,21.431737,7.858608


In [ ]:
attrib_data = marketing[[
    "date",
    "channel",
    "conversions",
    "spend",
    "spend_lag_1d",
    "spend_lag_3d",
    "spend_lag_7d"
]].dropna()


In [ ]:
from sklearn.linear_model import LinearRegression

attribution_results = []

for channel in attrib_data["channel"].unique():
    df = attrib_data[attrib_data["channel"] == channel]

    X = df[[
        "spend",
        "spend_lag_1d",
        "spend_lag_3d",
        "spend_lag_7d"
    ]]
    y = df["conversions"]

    model = LinearRegression()
    model.fit(X, y)

    attribution_results.append({
        "channel": channel,
        "same_day_weight": model.coef_[0],
        "lag_1d_weight": model.coef_[1],
        "lag_3d_weight": model.coef_[2],
        "lag_7d_weight": model.coef_[3]
    })


In [ ]:
attrib_weights = pd.DataFrame(attribution_results)
attrib_weights


,channel,same_day_weight,lag_1d_weight,lag_3d_weight,lag_7d_weight
0,Affiliate,0.012225,-0.000082,-0.000811,0.000270
1,Email,0.049609,-0.000570,0.002086,0.002052
2,Google Display,0.003167,0.000539,-0.000084,0.000273
3,Google Search,0.025988,-0.001706,-0.001175,0.003330
4,Influencer,0.004649,-0.000607,-0.000382,0.000069
5,Meta,0.008856,0.001079,-0.000186,0.001137


In [ ]:
attrib_weights["total_weight"] = (
    attrib_weights[[
        "same_day_weight",
        "lag_1d_weight",
        "lag_3d_weight",
        "lag_7d_weight"
    ]].sum(axis=1)
)

for col in ["same_day_weight", "lag_1d_weight", "lag_3d_weight", "lag_7d_weight"]:
    attrib_weights[col] = attrib_weights[col] / attrib_weights["total_weight"]


In [ ]:
baseline = baseline.merge(attrib_weights, on="channel", how="left")


In [ ]:
baseline["attrib_adjusted_roas"] = (
    baseline["avg_roas"] *
    (
        baseline["same_day_weight"] +
        baseline["lag_1d_weight"] +
        baseline["lag_3d_weight"] +
        baseline["lag_7d_weight"]
    )
)


In [ ]:
overall_attrib_roas = (
    baseline["total_revenue"].sum() /
    baseline["total_spend"].sum()
)


In [ ]:
def adjustment_factor_attrib(row, avg_roas):
    if row["attrib_adjusted_roas"] > avg_roas * 1.1:
        return 1.15
    elif row["attrib_adjusted_roas"] < avg_roas * 0.9:
        return 0.85
    else:
        return 1.0


In [ ]:
total_budget = baseline["total_spend"].sum()


In [ ]:
overall_attrib_roas = (
    baseline["total_revenue"].sum() /
    baseline["total_spend"].sum()
)


In [ ]:
def adjustment_factor(row, avg_roas):
    if row["attrib_adjusted_roas"] > avg_roas * 1.1:
        return 1.15   # strong performer → increase
    elif row["attrib_adjusted_roas"] < avg_roas * 0.9:
        return 0.85   # weak performer → reduce
    else:
        return 1.0    # neutral


In [ ]:
baseline["proposed_spend_raw"] = (
    baseline["total_spend"] *
    baseline.apply(adjustment_factor, axis=1, avg_roas=overall_attrib_roas)
)


In [ ]:
baseline["optimized_spend"] = (
    baseline["proposed_spend_raw"] /
    baseline["proposed_spend_raw"].sum()
) * total_budget


In [ ]:
baseline["spend_change_pct"] = (
    (baseline["optimized_spend"] - baseline["total_spend"])
    / baseline["total_spend"]
) * 100


In [ ]:
baseline["total_spend"].sum(), baseline["optimized_spend"].sum()


(np.float64(1167997724.4899998), np.float64(1167997724.4899998))

In [ ]:
channel_roles = {
    "Google Search": "Conversion",
    "Meta": "Discovery",
    "Influencer": "Awareness",
    "Google Display": "Awareness",
    "Affiliate": "Performance",
    "Email": "Retention"
}

baseline["channel_role"] = baseline["channel"].map(channel_roles)


In [ ]:
comparison = baseline.copy()

comparison["current_spend"] = comparison["total_spend"]
comparison["optimized_spend"] = comparison["optimized_spend"]

comparison["spend_change_pct"] = (
    (comparison["optimized_spend"] - comparison["current_spend"])
    / comparison["current_spend"]
) * 100

comparison = comparison[[
    "channel",
    "channel_role",
    "current_spend",
    "optimized_spend",
    "spend_change_pct",
    "avg_roas",
    "attrib_adjusted_roas"
]]

comparison


,channel,channel_role,current_spend,optimized_spend,spend_change_pct,avg_roas,attrib_adjusted_roas
0,Meta,Discovery,3.574055e+08,3.222087e+08,-9.847858,9.670158,9.670158
1,Google Search,Conversion,2.697092e+08,3.289658e+08,21.970545,32.411062,32.411062
2,Influencer,Awareness,1.797609e+08,1.620583e+08,-9.847858,7.771950,7.771950
3,Google Display,Awareness,1.492427e+08,1.345455e+08,-9.847858,3.000629,3.000629
4,Affiliate,Performance,1.200911e+08,1.082647e+08,-9.847858,13.695398,13.695398
5,Email,Retention,9.178837e+07,1.119548e+08,21.970545,66.795442,66.795442


In [ ]:
current_total_spend = baseline["total_spend"].sum()
current_total_revenue = baseline["total_revenue"].sum()

current_blended_roas = current_total_revenue / current_total_spend

current_blended_roas


np.float64(18.677411014550977)

In [ ]:
baseline["optimized_spend_share"] = (
    baseline["optimized_spend"] / baseline["optimized_spend"].sum()
)


In [ ]:
optimized_blended_roas = (
    baseline["optimized_spend_share"] *
    baseline["attrib_adjusted_roas"]
).sum()

optimized_blended_roas


np.float64(20.892134877299306)

In [ ]:
optimized_total_revenue = (
    optimized_blended_roas * current_total_spend
)

revenue_uplift_pct = (
    (optimized_total_revenue - current_total_revenue)
    / current_total_revenue
) * 100

optimized_total_revenue, revenue_uplift_pct


(np.float64(24401965996.423748), np.float64(11.857766909037366))

In [ ]:
impact_summary = pd.DataFrame({
    "Metric": [
        "Total Budget (₹ Cr)",
        "Blended ROAS (Before)",
        "Blended ROAS (After)",
        "Estimated Revenue (Before)",
        "Estimated Revenue (After)",
        "Estimated Revenue Uplift (%)"
    ],
    "Value": [
        round(current_total_spend / 1e7, 2),
        round(current_blended_roas, 2),
        round(optimized_blended_roas, 2),
        round(current_total_revenue / 1e7, 2),
        round(optimized_total_revenue / 1e7, 2),
        round(revenue_uplift_pct, 2)
    ]
})

impact_summary


,Metric,Value
0,Total Budget (₹ Cr),116.80
1,Blended ROAS (Before),18.68
2,Blended ROAS (After),20.89
3,Estimated Revenue (Before),2181.52
4,Estimated Revenue (After),2440.20
5,Estimated Revenue Uplift (%),11.86
